Import statement

In [228]:
import numpy as np
import boto3

dynamodb = boto3.resource("dynamodb")

Count how many row in Event Store

In [391]:
events_table = dynamodb.Table("marcostack-product_eventstore")
events_rows = events_table.scan()
events_rows["Count"]

1000

Count how many rows in log inspector

In [392]:
table = dynamodb.Table("marcostack-product-loginspector")
rows = table.scan()
count = rows["Count"]
count

1999

Aggregate logs by product id and use validator timestamp as start ts and handler as end ts

In [393]:
items =rows["Items"]
d = {}
for i in items:
    x = d.get(i["request_id"], {"start_ts": None, "end_ts": None})
    if "handler" in i["lambda_function"].lower():
        x["end_ts"] = i["timestamp"]
    else:
        x["start_ts"] = i["timestamp"]
    d.update({i["request_id"]:x})

d = dict(filter(lambda x:x[1].get("end_ts", None) != None and x[1].get("start_ts", None) != None, d.items()))
print("# of keys", len(d.keys()))
intervals = [int(x[1]["end_ts"]) - int(x[1]["start_ts"]) for x in d.items()]

# of keys 999


In [396]:

print("AVG: ", 0 if len(intervals) == 0 else sum(intervals)/len(intervals))
print("MAX: ", 0 if not intervals else max(intervals))
print("MIN: ", 0 if not intervals else min(intervals))

AVG:  3025.3303303303305
MAX:  4542
MIN:  738


In [397]:
a = np.array(intervals)
print("percentile 90", np.percentile(a, 90))
print("percentile 95", np.percentile(a, 95))
print("percentile 99", np.percentile(a, 99))

percentile 90 3452.4
percentile 95 3599.199999999999
percentile 99 4238.58


DELETE ALL DATA

In [387]:
print("Deleting... ", rows["Count"])
with table.batch_writer() as batch:
    for each in rows['Items']:
        batch.delete_item(
                 Key={
                 'request_id': each['request_id'],
                 'lambda_function': each['lambda_function']
                 }
             )
table.scan()["Count"]

Deleting...  2001


0

In [388]:
events_table = dynamodb.Table("marcostack-product_eventstore")
events_rows = events_table.scan()
print("Deleting... ", events_rows["Count"])
with events_table.batch_writer() as batch:
    for each in events_rows['Items']:
        batch.delete_item(
                 Key={
                 'aggregate_id': each['aggregate_id'],
                 'timestamp': each['timestamp']
                 }
             )
        
events_table.scan()["Count"]

Deleting...  1000


0